# Simple Network Dynamics simulator in Python

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import random
import scipy 
import networkx
import time as ttclock


# Set backend for matplotlib
matplotlib.use('nbagg')


def initialize_network(population_size, link_probability, initial_infected_ratio):
    """Initialize the network"""
    network = networkx.erdos_renyi_graph(population_size, link_probability)
    positions = networkx.random_layout(network)
    for i in network.nodes():
        if random.random() < initial_infected_ratio:
            network.nodes[i]['state'] = infected
        else:
            network.nodes[i]['state'] = susceptible

    return network, positions

def draw(ax, network, positions, iteration=0):
    networkx.draw(network, pos = positions,
                  node_color = [network.nodes[i]['state'] for i in network.nodes()],
                  with_labels = False, edge_color = 'c', vmin = 0, vmax = 1)
    ax.set_title('t = {}'.format(iteration))
    display(fig)
    clear_output(wait = True)
    plt.pause(0.5)
    

# Set the properties of the network
random.seed()
population_size = 50
link_probability = 0.05    
initial_infected_ratio = 0.2
infection_probability = 0.2
recovery_probability = 0.5
susceptible = 0
infected = 1

network, positions = initialize_network(population_size, link_probability, initial_infected_ratio)
fig = plt.figure(figsize=(8.0, 8.0))
ax = fig.add_subplot()
draw(ax, network, positions)

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

def step(network, iteration, infection_probability, recovery_probability, susceptible=0, infected=1):
    iteration += 1
    next_network = network.copy()
    for i in network.nodes():
        if network.nodes[i]['state'] == susceptible:
            next_network.nodes[i]['state'] = susceptible
            for j in network.neighbors(i):
                if network.nodes[j]['state'] == infected:
                    if random.random() < infection_probability:
                        next_network.nodes[i]['state'] = infected
                        break
        else:
            if random.random() < recovery_probability:
                next_network.nodes[i]['state'] = susceptible
            else:
                next_network.nodes[i]['state'] = infected

    return next_network

def run(network, positions, infection_probability, recovery_probability, niterations=50, sleep_time=1):
    total_infected = []
    total_susceptible = []
    iteration = 0
    fig = plt.figure(figsize=(8.0, 8.0))
    ax = fig.add_subplot(1,1,1)
    persons_infected = [n for n in network.nodes() if network.nodes[n]['state'] == 1]
    total_infected.append(len(persons_infected))
    while iteration < niterations:
        iteration += 1
        next_network = step(network, iteration, infection_probability, recovery_probability)
        network = next_network.copy()
        persons_infected = [n for n in network.nodes if network.nodes[n]['state'] == 1]
        total_infected.append(len(persons_infected))
        ax.cla()
        networkx.draw(network, pos = positions,
                  node_color = [network.nodes[i]['state'] for i in network.nodes()],
                  with_labels = False, edge_color = 'c', vmin = 0, vmax = 1, ax=ax)
        ax.set_title('t = {}'.format(iteration))
        display(fig)
        clear_output(wait = True)
        plt.pause(sleep_time)
    return total_infected
        

total_infected = run(network, positions, infection_probability, recovery_probability)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot() 

x = np.arange(0, len(total_infected), 1)
y = np.array(total_infected)
  
ax.plot(x, y)